# Agentic AI with CrewAI + Accelerator RAG Tool

This optional notebook shows how to build a simple CrewAI-based agent that uses:

- The **accelerator RAG API** as a tool.
- A **calculator tool** for arithmetic.

It mirrors the logic from `agent_watsonx.ipynb`, but expressed with CrewAI abstractions.


In [ ]:
# Install CrewAI (run once per environment)
!pip install -q crewai crewai-tools requests


In [ ]:
import os
import json
import time
import ast
import operator as op
from typing import Any

import requests
from crewai import Agent, Task, Crew, Process
from crewai_tools import tool

ACCELERATOR_API_URL = os.getenv("ACCELERATOR_API_URL", "http://localhost:8000/ask")


In [ ]:
@tool("rag_service_tool")
def rag_service_tool(question: str) -> str:
    """Call the accelerator `/ask` endpoint to answer the user's question.

    Returns a formatted string containing answer and citations (if any).
    """
    payload = {"question": question}
    resp = requests.post(ACCELERATOR_API_URL, json=payload, timeout=60)
    resp.raise_for_status()
    data = resp.json()
    answer = data.get("answer") or data.get("result") or "(no answer field)"
    citations = data.get("citations") or data.get("chunks") or []
    return f"ANSWER: {answer}\nCITATIONS: {citations}"


_allowed_operators = {
    ast.Add: op.add,
    ast.Sub: op.sub,
    ast.Mult: op.mul,
    ast.Div: op.truediv,
    ast.Pow: op.pow,
}


def _eval_ast(node):
    if isinstance(node, ast.Num):  # type: ignore[attr-defined]
        return node.n
    if isinstance(node, ast.BinOp) and type(node.op) in _allowed_operators:
        return _allowed_operators[type(node.op)](_eval_ast(node.left), _eval_ast(node.right))
    if isinstance(node, ast.UnaryOp) and isinstance(node.op, (ast.UAdd, ast.USub)):
        value = _eval_ast(node.operand)
        return +value if isinstance(node.op, ast.UAdd) else -value
    raise ValueError("Unsupported expression")


@tool("calculator_tool")
def calculator_tool(expression: str) -> str:
    """Safely evaluate an arithmetic expression like `2 * (3 + 4)`"""
    try:
        parsed = ast.parse(expression, mode="eval")
        result = _eval_ast(parsed.body)
        return str(result)
    except Exception as e:
        return f"Error evaluating expression: {e}"


## Define the CrewAI Agent & Task

We create a single agent that can decide which tool to use, guided by its instructions.


In [ ]:
support_agent = Agent(
    role="Workshop Support Agent",
    goal=(
        "Help users with questions about the workshop, using the RAG service for factual questions "
        "and the calculator tool for math."
    ),
    backstory=(
        "You are an AI assistant built for this workshop. "
        "When the question is clearly about numbers or arithmetic, prefer the calculator tool. "
        "Otherwise, use the rag_service_tool for grounded answers."
    ),
    tools=[rag_service_tool, calculator_tool],
    verbose=True,
)

support_task = Task(
    description=(
        "Answer the user's question using your tools. "
        "Explain which tool you used and summarize the result."
    ),
    expected_output="A helpful, grounded answer with a short explanation.",
    agent=support_agent,
)

crew = Crew(
    agents=[support_agent],
    tasks=[support_task],
    process=Process.sequential,
)


## Run a Few Example Questions


In [ ]:
questions = [
    "What is Retrieval-Augmented Generation?",
    "Compute 3 * (5 + 7).",
]

for q in questions:
    print("=" * 80)
    print("USER:", q)
    result = crew.kickoff(inputs={"input": q})
    print("AGENT OUTPUT:\n", result)
